# Univariate Kalman Filter
$x$: state of the system <br>
$P$: uncertainty in the system (Initialized at the start and updated in the process) <br>
$u$: movement <br>
$Q$: uncertainty in the movement <br>
$z$: measurements <br> 
$R$: uncertainty in the measurements <br>


<u>Predict</u>

$\begin{array}{|l|l|l|}
\hline
\text{Equation} & \text{Implementation} & \text{Kalman Form}\\
\hline
 \bar x = x + f_x & \bar\mu = \mu + \mu_{f_x} & \bar x = x + dx\\
& \bar\sigma^2 = \sigma^2 + \sigma_{f_x}^2 & \bar P = P + Q\\
\hline
\end{array}$

<u>Update</u>

$\begin{array}{|l|l|l|}
\hline
\text{Equation} & \text{Implementation}& \text{Kalman Form}\\
\hline
 x = \| \mathcal L\bar x\| & y = z - \bar\mu & y = z - \bar x\\
 & K = \frac {\bar\sigma^2} {\bar\sigma^2 + \sigma_z^2} & K = \frac {\bar P}{\bar P+R}\\
 & \mu = \bar \mu + Ky & x = \bar x + Ky\\
 & \sigma^2 = \frac {\bar\sigma^2 \sigma_z^2} {\bar\sigma^2 + \sigma_z^2} & P = (1-K)\bar P\\
\hline
\end{array}$

In [126]:
# Imports
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
plt.style.use('seaborn')

In [161]:
# Multiplication and convolution of two gaussians
def add_gaussians(x, P, u, Q):
    return (x+u, P+Q)
def multiply_gaussians(z, R, x, P):
    kalman_gain = P/(P+R)
    residual = z - x
    return (x + kalman_gain*residual, (P*R)/(P+R))   

In [165]:
# Interactive kalman filtering
@interact(N = (1,20,1), R = (0,3,0.5), u=(-2,2,1), Q=(1,10,1))
def kalman_filtering(N,R,u,Q):
    x0 = 0 
    P = 100
    np.random.seed(123)
    z_all = np.arange(N) + R* np.random.randn(N)
    x_pred = []
    x_est = []
    
    for i, z in enumerate(z_all):
        if i == 0:
            x_pred.append(add_gaussians(x0,P,u,Q))
            x_est.append(multiply_gaussians(z, R, x_pred[i][0], P))
            
        else:
            x_pred.append(add_gaussians(x_est[i-1][0], #  Estimated mean
                                        x_est[i-1][1], #  Estimated variance
                                        u,Q))
            x_est.append(multiply_gaussians(z,R,
                                            x_pred[i][0], 
                                            x_pred[i][1]))
                                        
    t = range(N)
    x_pred_mean = [pred[0] for pred in x_pred]
    x_pred_var = [pred[1] for pred in x_pred]
    x_est_mean = [est[0] for est in x_est]
    x_est_var = [est[1] for est in x_est]
    
    plt.scatter(t, z_all, c='r', label = '$z$')
    plt.plot(t, x_pred_mean, c='b', label = '$x (pred)$')
    plt.plot(t, x_est_mean, c='k', label='$x$')
    plt.errorbar(t, x_est_mean, yerr = x_est_var, 
                 c='k', fmt='o', capthick=2, capsize=10, 
                 alpha=0.5)
    plt.xlabel('t')
    plt.legend()
        

interactive(children=(IntSlider(value=10, description='N', max=20, min=1), FloatSlider(value=1.0, description=…

# Multivariate Kalman Filter
$\mathbf{x}$: State. Contains quantities such as position, velocity etc. <br>
$\mathbf{P}$: Initial covariace. gets updated in the process <br>
$\mathbf{F}$: State transision function/matrix. <br>
$\mathbf{Q}$: Noise in the process <br>
$\mathbf{u}$: Control Input (optional)<br>
$\mathbf{B}$: Control function/matrix (optional)<br>
$\mathbf{H}$: Measurement function/matrix - Converts a state into a measurement to calculate the residual. <br>
$\mathbf{z}$: Measurement vector <br>
$\mathbf{R}$: Measurement Noise matrix <br>

<u>**Predict**</u>

$\begin{array}{|l|l|l|}
\hline
\text{Univariate} & \text{Univariate} & \text{Multivariate}\\
& \text{(Kalman form)} & \\
\hline
\bar \mu = \mu + \mu_{f_x} & \bar x = x + dx & \bar{\mathbf x} = \mathbf{Fx} + \mathbf{Bu}\\
\bar\sigma^2 = \sigma_x^2 + \sigma_{f_x}^2 & \bar P = P + Q & \bar{\mathbf P} = \mathbf{FPF}^\mathsf T + \mathbf Q \\
\hline
\end{array}$

#### Prediction: Summary

Your job as a designer is to specify the matrices for

* $\mathbf x$, $\mathbf P$: the state and covariance
* $\mathbf F$,  $\mathbf Q$: the process model and noise covariance
* $\mathbf{B,u}$: Optionally, the control input and function





<u>**Update**</u>

$\begin{array}{|l|l|l|}
\hline
\text{Univariate} & \text{Univariate} & \text{Multivariate}\\
& \text{(Kalman form)} & \\
\hline
& y = z - \bar x & \mathbf y = \mathbf z - \mathbf{H\bar x} \\
& K = \frac{\bar P}{\bar P+R}&
\mathbf K = \mathbf{\bar{P}H}^\mathsf T (\mathbf{H\bar{P}H}^\mathsf T + \mathbf R)^{-1} \\
\mu=\frac{\bar\sigma^2\, \mu_z + \sigma_z^2 \, \bar\mu} {\bar\sigma^2 + \sigma_z^2} & x = \bar x + Ky & \mathbf x = \bar{\mathbf x} + \mathbf{Ky} \\
\sigma^2 = \frac{\sigma_1^2\sigma_2^2}{\sigma_1^2+\sigma_2^2} & P = (1-K)\bar P &
\mathbf P = (\mathbf I - \mathbf{KH})\mathbf{\bar{P}} \\
\hline
\end{array}$